### GWANW 2022 Bilby tutorial

This notebook walks you through configuring and running a GW parameter estimation job using the Bayesian inference package `Bilby`. This tutorial is based on this example, which looked at a different event: https://git.ligo.org/lscsoft/bilby/blob/master/examples/gw_examples/data_examples/GW150914.py

First we'll make sure we have installed the required Python packages

In [2]:
import sys
!{sys.executable} -m pip install matplotlib gwosc gwpy bilby ipywidgets

import math
import matplotlib.pyplot as plt
import bilby
from gwpy.timeseries import TimeSeries
from gwosc import datasets

For this tutorial we'll analyze one of the more notable events from the last observing run (O3), GW190521. This signal originated from a binary system of two black holes with masses of approximately 85 and 65 Msun, which merged to form a 142 Msun remnant black hole.

In [4]:
# Get trigger time of the event
trigger_time = datasets.event_gps("GW190521")

# Analyze 4s of data around the trigger time
seglen = 4
post_trigger = 2
end_time = trigger_time + post_trigger
start_time = end_time - seglen

# Frequency band
flow = 11
fhigh = 512

# Options for PSD calculation
psd_seglen = 32 * seglen
psd_end_time = start_time
psd_start_time = start_time - psd_seglen
roll_off = 0.2
overlap = 0

# Detector list
detectors = ["H1", "L1", "V1"]

Next we fetch the strain data to be analyzed. We can do this using GWPy (see Adrian's tutorial)

In [3]:
ifo_list = bilby.gw.detector.InterferometerList([])
for det in detectors:
    print(f"Fetching data for {det}")
    strain_data = TimeSeries.fetch_open_data(det, start_time, end_time)
    
    # Insert data into bilby interferometer class
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    ifo.strain_data.set_from_gwpy_timeseries(strain_data)
    
    # Calculate the PSD. There is more than one way of doing this. The method shown
    # here involves looking at a long stretch of data before the analysis segment,
    # splitting it into several chunks, taking the spectrum of each and averaging
    # over all segments.
    psd_data = TimeSeries.fetch_open_data(det, psd_start_time, psd_end_time)
    psd_alpha = 2 * roll_off / seglen
    psd = psd_data.psd(
        fftlength=seglen, overlap=overlap, window=("tukey", psd_alpha), method="median")
    
    # Assign the PSD to the detector
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=psd.frequencies.value, psd_array=psd.value)
    
    ifo.maximum_frequency = fhigh
    ifo.minimum_frequency = flow
    ifo_list.append(ifo)

Fetching data for H1
Fetching data for L1
Fetching data for V1


Set the priors. `Bilby` includes several prior distributions that we can use, or we can also define our own.

For this example, we first load the default BBH priors, and then tailor the ranges to this particular event. In the interest of time, we will run a "minimal" example by fixing most of the parameters and sampling in just a few, but in actual analyses we usually allow all parameters to float.

In [4]:
priors = bilby.gw.prior.BBHPriorDict()

# Sample in chirp mass and mass ratio with
# priors that give uniform distributions in the component masses
priors.pop("mass_1")
priors.pop("mass_2")
priors["chirp_mass"] = bilby.gw.prior.UniformInComponentsChirpMass(
    minimum=70, maximum=180, name="chirp_mass", latex_label=r"$\mathcal{M}$")
priors["mass_ratio"] = bilby.gw.prior.UniformInComponentsMassRatio(
    minimum=0.1, maximum=1, name="mass_ratio", latex_label=r"$q$")

# Luminosity distance prior is uniform in comoving source-frame volume
priors["luminosity_distance"] = bilby.gw.prior.UniformSourceFrame(
    minimum=1e2, maximum=1e5, name="luminosity_distance", latex_label=r"$d_L$", unit="Mpc")

# Set time prior to be +/- 0.1s around the trigger time
priors["geocent_time"] = bilby.core.prior.Uniform(
    minimum=trigger_time - 0.1, maximum=trigger_time + 0.1, name="geocent_time", latex_label=r"$t_{\rm c}$")

# Fix the remaining parameters to some appropriate 
priors["a_1"] = 0.4
priors["a_2"] = 0.42
priors["tilt_1"] = 0
priors["tilt_2"] = math.pi
priors["phi_12"] = 0
priors["phi_jl"] = 0
priors["psi"] = 3.13
priors["ra"] = 3.37
priors["dec"] = 0.46
priors["theta_jn"] = 0.93

16:07 bilby INFO    : No prior given, using default BBH priors in /opt/anaconda3/envs/py39/lib/python3.9/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


Create the likelihood function for our analysis. The signal model is defined through the `waveform_generator` class, where we can set the approximant that will be used to reconstruct the signal. Then we pass this along with the priors defined above into a `GravitationalWaveTransient` likelihood.

For this specific case, the likelihood is already marginalized over time and distance, so ultimately we search over chirp mass and mass ratio.

In [5]:
waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={"waveform_approximant": "IMRPhenomD",
                        "reference_frequency": 20, 
                        "minimum_frequency": flow})

likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list, waveform_generator, priors=priors,
    time_marginalization=True,
    phase_marginalization=True,
    distance_marginalization=True)

16:07 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
16:07 bilby INFO    : Loaded distance marginalisation lookup table from .distance_marginalization_lookup.npz.


With everything set up, we can run the analysis. There is a helpful progress bar that you can monitor. In particular, take note of `dlogz`. This is an estimate of the total remaining evidence in the parameter space, which decreases (roughly) monotonically as the run progresses. The run terminates when `dlogz` drops below some threshold (default 0.1). The run will take about 2 hours to complete on your local machine. 

In [6]:
outdir = "GW190521_minimal"
label = "GW190521_minimal"
bilby.core.utils.check_directory_exists_and_if_not_mkdir(outdir)

result = bilby.run_sampler(
    likelihood=likelihood, priors=priors,
    outdir=outdir, label=label,
    sampler="dynesty", nlive=1000, check_point_delta_t=600, 
    check_point_plot=True, npool=1,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
    result_class=bilby.gw.result.CBCResult)

# Output plots
result.plot_corner(["chirp_mass", "mass_ratio"])
result.plot_waveform_posterior(interferometers=ifo_list)

16:07 bilby INFO    : Running for label 'GW190521_minimal', output will be saved to 'GW190521_minimal'
16:07 bilby INFO    : Using lal version 7.1.3
16:07 bilby INFO    : Using lal git version Branch: None;Tag: lal-v7.1.3;Id: fa9914e5d72cc6168463b5fce79eeba8037404ad;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
16:07 bilby INFO    : Using lalsimulation version 3.0.0
16:07 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsimulation-v3.0.0;Id: e8e5ff5aa1ae2676019f59b6785f55284c417fda;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
16:07 bilby INFO    : Search parameters:
16:07 bilby INFO    :   mass_ratio = UniformInComponentsMassRatio(minimum=0.1, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None)
16:07 bilby INFO    :   chirp_mass = UniformInComponentsChirpMass(minimum=70, maximum=180, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=N

0it [00:00, ?it/s]

16:07 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'rwalk', 'verbose': True, 'periodic': None, 'reflective': None, 'check_point_delta_t': 1800, 'nlive': 1000, 'first_update': None, 'walks': 100, 'npdim': None, 'rstate': None, 'queue_size': 1, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': 1.5, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 8.0, 'facc': 0.2, 'slices': 5, 'update_interval': 600, 'print_func': <bound method Dynesty._print_func of <bilby.core.sampler.dynesty.Dynesty object at 0x7fc1ce822df0>>, 'dlogz': 0.1, 'maxiter': None, 'maxcall': None, 'logl_max': inf, 'add_live': True, 'print_progress': True, 'save_bounds': False, 'n_effective': None, 'maxmcmc': 5000, 'nact': 5, 'print_method': 'tqdm'}
16:07 bilby INFO    : Checkpoint every check_point_delta_t = 600s
16:07 bilby INFO    : Using dynesty version 1.0.1
16:07 bilby INFO    : Using the bi

17:14 bilby INFO    : Sampling time: 1:06:48.672906
17:14 bilby INFO    : Reconstructing marginalised parameters.


  0%|          | 0/7586 [00:00<?, ?it/s]

17:19 bilby INFO    : Generating sky frame parameters.


  0%|          | 0/7586 [00:00<?, ?it/s]

17:19 bilby INFO    : Computing SNRs for every sample.


  0%|          | 0/7586 [00:00<?, ?it/s]

17:20 bilby INFO    : Summary of results:
nsamples: 7586
ln_noise_evidence: -6392.176
ln_evidence: -6289.936 +/-  0.081
ln_bayes_factor: 102.240 +/-  0.081

17:20 bilby INFO    : Generating waveform figure for H1
17:20 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
17:20 bilby INFO    : Generating waveform figure for L1
17:20 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
17:20 bilby INFO    : Generating waveform figure for V1
17:20 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_convers